In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('Dataset/test.csv')

In [4]:
df.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7


In [5]:
df1 = pd.read_csv('Dataset/train.csv')

In [6]:
df1.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [7]:
targets = df1[['pet_id','breed_category','pet_category']]

In [8]:
df1 = df1.drop('breed_category', axis = True)

In [9]:
df1= df1.drop('pet_category', axis = True)

In [10]:
df1.shape

(18834, 9)

In [11]:
df.shape

(8072, 9)

In [12]:
df1.loc[18833:18835]

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
18833,ANSL_73558,2011-12-13 00:00:00,2018-10-26 14:18:00,0.0,Brown,0.86,37.4,0,1


In [13]:
df.loc[0:1]

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1


In [14]:
df.loc[8071:]

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
8071,ANSL_69436,2016-02-02 00:00:00,2017-02-28 16:47:00,0.0,Black,0.64,24.51,0,1


In [15]:
test = pd.concat([df1,df])

In [16]:
test['issue_date'] = pd.to_datetime(test['issue_date'])

In [17]:
test['listing_date'] = pd.to_datetime(test['listing_date'])

In [18]:
test['days'] =  (test['listing_date'] - test['issue_date']).dt.days

In [19]:
test['days'].max()

9154

In [20]:
test['days']=test['days'].mask(test['days']<0).fillna(test['days'].mean())

In [21]:
test['month'] = test['days']/30

In [22]:
test['month'].min()

0.6333333333333333

In [23]:
test[18833:18836]

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,days,month
18833,ANSL_73558,2011-12-13,2018-10-26 14:18:00,0.0,Brown,0.86,37.40,0,1,2509.0,83.633333
0,ANSL_75005,2005-08-17,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7,4404.0,146.800000
1,ANSL_76663,2018-11-15,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1,174.0,5.800000


In [24]:
test['condition'].fillna(3.0, inplace=True)

In [25]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    return(res)

In [26]:
test = encode_and_bind(test, 'color_type')

In [27]:
test.drop('color_type',axis = 1, inplace = True)

In [28]:
test['height(m)']=test['height(cm)'].apply(lambda x:x*(.01))

In [29]:
test

,pet_id,issue_date,listing_date,condition,length(m),height(cm),X1,X2,days,month,...,color_type_Silver Tabby,color_type_Tan,color_type_Torbie,color_type_Tortie,color_type_Tortie Point,color_type_Tricolor,color_type_White,color_type_Yellow,color_type_Yellow Brindle,height(m)
0,ANSL_69903,2016-07-10,2016-09-21 16:25:00,2.0,0.80,7.78,13,9,73.0,2.433333,...,0,0,0,0,0,0,0,0,0,0.0778
1,ANSL_66892,2013-11-21,2018-12-27 17:47:00,1.0,0.72,14.19,13,9,1862.0,62.066667,...,0,0,0,0,0,0,1,0,0,0.1419
2,ANSL_69750,2014-09-28,2016-10-19 08:24:00,3.0,0.15,40.90,15,4,752.0,25.066667,...,0,0,0,0,0,0,0,0,0,0.4090
3,ANSL_71623,2016-12-31,2019-01-25 18:30:00,1.0,0.62,17.82,0,1,755.0,25.166667,...,0,0,0,0,0,0,1,0,0,0.1782
4,ANSL_57969,2017-09-28,2017-11-19 09:38:00,2.0,0.50,11.06,18,4,52.0,1.733333,...,0,0,0,0,0,0,0,0,0,0.1106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8067,ANSL_66809,2016-02-10,2017-03-10 14:56:00,2.0,0.82,36.08,13,9,394.0,13.133333,...,0,0,0,0,0,0,0,0,0,0.3608
8068,ANSL_59041,2015-12-07,2018-02-12 00:00:00,0.0,0.49,27.54,13,9,798.0,26.600000,...,0,1,0,0,0,0,0,0,0,0.2754
8069,ANSL_60034,2015-12-08,2017-01-04 17:19:00,0.0,0.98,37.19,0,7,393.0,13.100000,...,0,0,0,0,0,0,0,0,0,0.3719
8070,ANSL_58066,2016-06-28,2017-07-20 18:19:00,3.0,0.79,23.83,0,2,387.0,12.900000,...,0,0,0,0,0,0,0,0,0,0.2383


In [30]:
test['height(m)']=test['height(m)'].mask(test['height(m)']==0).fillna(test['height(m)'].mean())

In [31]:
test['length(m)']=test['length(m)'].mask(test['length(m)']==0).fillna(test['length(m)'].mean())

In [32]:
test['X1']=test['X1'].mask(test['X1']==0).fillna(test['X1'].median())

In [33]:
test['X2']=test['X2'].mask(test['X2']==0).fillna(test['X2'].median())

In [34]:
test

,pet_id,issue_date,listing_date,condition,length(m),height(cm),X1,X2,days,month,...,color_type_Silver Tabby,color_type_Tan,color_type_Torbie,color_type_Tortie,color_type_Tortie Point,color_type_Tricolor,color_type_White,color_type_Yellow,color_type_Yellow Brindle,height(m)
0,ANSL_69903,2016-07-10,2016-09-21 16:25:00,2.0,0.80,7.78,13.0,9.0,73.0,2.433333,...,0,0,0,0,0,0,0,0,0,0.0778
1,ANSL_66892,2013-11-21,2018-12-27 17:47:00,1.0,0.72,14.19,13.0,9.0,1862.0,62.066667,...,0,0,0,0,0,0,1,0,0,0.1419
2,ANSL_69750,2014-09-28,2016-10-19 08:24:00,3.0,0.15,40.90,15.0,4.0,752.0,25.066667,...,0,0,0,0,0,0,0,0,0,0.4090
3,ANSL_71623,2016-12-31,2019-01-25 18:30:00,1.0,0.62,17.82,0.0,1.0,755.0,25.166667,...,0,0,0,0,0,0,1,0,0,0.1782
4,ANSL_57969,2017-09-28,2017-11-19 09:38:00,2.0,0.50,11.06,18.0,4.0,52.0,1.733333,...,0,0,0,0,0,0,0,0,0,0.1106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8067,ANSL_66809,2016-02-10,2017-03-10 14:56:00,2.0,0.82,36.08,13.0,9.0,394.0,13.133333,...,0,0,0,0,0,0,0,0,0,0.3608
8068,ANSL_59041,2015-12-07,2018-02-12 00:00:00,0.0,0.49,27.54,13.0,9.0,798.0,26.600000,...,0,1,0,0,0,0,0,0,0,0.2754
8069,ANSL_60034,2015-12-08,2017-01-04 17:19:00,0.0,0.98,37.19,0.0,7.0,393.0,13.100000,...,0,0,0,0,0,0,0,0,0,0.3719
8070,ANSL_58066,2016-06-28,2017-07-20 18:19:00,3.0,0.79,23.83,0.0,2.0,387.0,12.900000,...,0,0,0,0,0,0,0,0,0,0.2383


In [35]:
test.drop('days',axis = 1, inplace = True)

In [36]:
train_df=test[:18834]

In [37]:
test_df= test[18834:]

In [38]:
train_df.shape

(18834, 66)

In [39]:
test_df.shape

(8072, 66)

In [40]:
train_df = pd.concat([train_df,targets], axis =1)

In [41]:
train_df.head()

,pet_id,issue_date,listing_date,condition,length(m),height(cm),X1,X2,month,color_type_Agouti,...,color_type_Tortie,color_type_Tortie Point,color_type_Tricolor,color_type_White,color_type_Yellow,color_type_Yellow Brindle,height(m),pet_id,breed_category,pet_category
0,ANSL_69903,2016-07-10,2016-09-21 16:25:00,2.0,0.80,7.78,13.0,9.0,2.433333,0,...,0,0,0,0,0,0,0.0778,ANSL_69903,0.0,1
1,ANSL_66892,2013-11-21,2018-12-27 17:47:00,1.0,0.72,14.19,13.0,9.0,62.066667,0,...,0,0,0,1,0,0,0.1419,ANSL_66892,0.0,2
2,ANSL_69750,2014-09-28,2016-10-19 08:24:00,3.0,0.15,40.90,15.0,4.0,25.066667,0,...,0,0,0,0,0,0,0.4090,ANSL_69750,2.0,4
3,ANSL_71623,2016-12-31,2019-01-25 18:30:00,1.0,0.62,17.82,0.0,1.0,25.166667,0,...,0,0,0,1,0,0,0.1782,ANSL_71623,0.0,2
4,ANSL_57969,2017-09-28,2017-11-19 09:38:00,2.0,0.50,11.06,18.0,4.0,1.733333,0,...,0,0,0,0,0,0,0.1106,ANSL_57969,0.0,1


In [42]:
test_df.head()

,pet_id,issue_date,listing_date,condition,length(m),height(cm),X1,X2,month,color_type_Agouti,...,color_type_Silver Tabby,color_type_Tan,color_type_Torbie,color_type_Tortie,color_type_Tortie Point,color_type_Tricolor,color_type_White,color_type_Yellow,color_type_Yellow Brindle,height(m)
0,ANSL_75005,2005-08-17,2017-09-07 15:35:00,0.0,0.87,42.73,0.0,7.0,146.800000,0,...,0,0,0,0,0,0,0,0,0,0.4273
1,ANSL_76663,2018-11-15,2019-05-08 17:24:00,1.0,0.06,6.71,0.0,1.0,5.800000,0,...,0,0,0,0,0,0,0,0,0,0.0671
2,ANSL_58259,2012-10-11,2018-04-02 16:51:00,1.0,0.24,41.21,0.0,7.0,66.633333,0,...,0,0,0,0,0,0,0,0,0,0.4121
3,ANSL_67171,2015-02-13,2018-04-06 07:25:00,1.0,0.29,8.46,7.0,1.0,38.266667,0,...,0,0,0,0,0,0,0,0,0,0.0846
4,ANSL_72871,2017-01-18,2018-04-26 13:42:00,1.0,0.71,30.92,0.0,7.0,15.433333,0,...,0,0,0,0,0,0,0,0,0,0.3092


In [43]:
train_df.drop('height(cm)', axis = 1, inplace = True)

In [44]:
test_df.drop('height(cm)', axis = 1, inplace = True)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/frame.py:4110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [45]:
train_df.columns.values

array(['pet_id', 'issue_date', 'listing_date', 'condition', 'length(m)',
       'X1', 'X2', 'month', 'color_type_Agouti', 'color_type_Apricot',
       'color_type_Black', 'color_type_Black Brindle',
       'color_type_Black Smoke', 'color_type_Black Tabby',
       'color_type_Black Tiger', 'color_type_Blue',
       'color_type_Blue Cream', 'color_type_Blue Merle',
       'color_type_Blue Point', 'color_type_Blue Smoke',
       'color_type_Blue Tabby', 'color_type_Blue Tick',
       'color_type_Blue Tiger', 'color_type_Brown',
       'color_type_Brown Brindle', 'color_type_Brown Merle',
       'color_type_Brown Tabby', 'color_type_Brown Tiger',
       'color_type_Buff', 'color_type_Calico', 'color_type_Calico Point',
       'color_type_Chocolate', 'color_type_Chocolate Point',
       'color_type_Cream', 'color_type_Cream Tabby', 'color_type_Fawn',
       'color_type_Flame Point', 'color_type_Gold', 'color_type_Gray',
       'color_type_Gray Tabby', 'color_type_Green',
       'color_type

In [46]:
train_df


,pet_id,issue_date,listing_date,condition,length(m),X1,X2,month,color_type_Agouti,color_type_Apricot,...,color_type_Tortie,color_type_Tortie Point,color_type_Tricolor,color_type_White,color_type_Yellow,color_type_Yellow Brindle,height(m),pet_id,breed_category,pet_category
0,ANSL_69903,2016-07-10,2016-09-21 16:25:00,2.0,0.80,13.0,9.0,2.433333,0,0,...,0,0,0,0,0,0,0.0778,ANSL_69903,0.0,1
1,ANSL_66892,2013-11-21,2018-12-27 17:47:00,1.0,0.72,13.0,9.0,62.066667,0,0,...,0,0,0,1,0,0,0.1419,ANSL_66892,0.0,2
2,ANSL_69750,2014-09-28,2016-10-19 08:24:00,3.0,0.15,15.0,4.0,25.066667,0,0,...,0,0,0,0,0,0,0.4090,ANSL_69750,2.0,4
3,ANSL_71623,2016-12-31,2019-01-25 18:30:00,1.0,0.62,0.0,1.0,25.166667,0,0,...,0,0,0,1,0,0,0.1782,ANSL_71623,0.0,2
4,ANSL_57969,2017-09-28,2017-11-19 09:38:00,2.0,0.50,18.0,4.0,1.733333,0,0,...,0,0,0,0,0,0,0.1106,ANSL_57969,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18829,ANSL_51738,2017-01-26,2018-03-09 15:35:00,2.0,0.44,0.0,1.0,13.566667,0,0,...,0,0,1,0,0,0,0.2736,ANSL_51738,0.0,2
18830,ANSL_59900,2016-06-18,2017-07-09 08:37:00,3.0,0.73,15.0,4.0,12.866667,0,0,...,0,0,0,0,0,0,0.1425,ANSL_59900,2.0,4
18831,ANSL_53210,2010-07-21,2018-08-22 14:27:00,0.0,0.99,13.0,9.0,98.466667,0,0,...,0,0,0,0,0,0,0.2813,ANSL_53210,1.0,1
18832,ANSL_63468,2017-05-12,2018-02-08 14:05:00,0.0,0.55,13.0,9.0,9.066667,0,0,...,0,0,0,0,0,0,0.4482,ANSL_63468,1.0,2


In [47]:
train_df.drop('issue_date', axis = 1, inplace = True)
train_df.drop('listing_date', axis = 1, inplace=True)
test_df.drop('issue_date', axis = 1, inplace = True)
test_df.drop('listing_date', axis = 1, inplace=True)

In [48]:
train_df

,pet_id,condition,length(m),X1,X2,month,color_type_Agouti,color_type_Apricot,color_type_Black,color_type_Black Brindle,...,color_type_Tortie,color_type_Tortie Point,color_type_Tricolor,color_type_White,color_type_Yellow,color_type_Yellow Brindle,height(m),pet_id,breed_category,pet_category
0,ANSL_69903,2.0,0.80,13.0,9.0,2.433333,0,0,0,0,...,0,0,0,0,0,0,0.0778,ANSL_69903,0.0,1
1,ANSL_66892,1.0,0.72,13.0,9.0,62.066667,0,0,0,0,...,0,0,0,1,0,0,0.1419,ANSL_66892,0.0,2
2,ANSL_69750,3.0,0.15,15.0,4.0,25.066667,0,0,0,0,...,0,0,0,0,0,0,0.4090,ANSL_69750,2.0,4
3,ANSL_71623,1.0,0.62,0.0,1.0,25.166667,0,0,0,0,...,0,0,0,1,0,0,0.1782,ANSL_71623,0.0,2
4,ANSL_57969,2.0,0.50,18.0,4.0,1.733333,0,0,1,0,...,0,0,0,0,0,0,0.1106,ANSL_57969,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18829,ANSL_51738,2.0,0.44,0.0,1.0,13.566667,0,0,0,0,...,0,0,1,0,0,0,0.2736,ANSL_51738,0.0,2
18830,ANSL_59900,3.0,0.73,15.0,4.0,12.866667,0,0,0,0,...,0,0,0,0,0,0,0.1425,ANSL_59900,2.0,4
18831,ANSL_53210,0.0,0.99,13.0,9.0,98.466667,0,0,0,0,...,0,0,0,0,0,0,0.2813,ANSL_53210,1.0,1
18832,ANSL_63468,0.0,0.55,13.0,9.0,9.066667,0,0,0,0,...,0,0,0,0,0,0,0.4482,ANSL_63468,1.0,2


In [49]:
train_df=train_df.drop(df.columns[0], axis=1)

In [50]:
train_df.loc[train_df['length(m)'] == 0]

,condition,length(m),X1,X2,month,color_type_Agouti,color_type_Apricot,color_type_Black,color_type_Black Brindle,color_type_Black Smoke,...,color_type_Torbie,color_type_Tortie,color_type_Tortie Point,color_type_Tricolor,color_type_White,color_type_Yellow,color_type_Yellow Brindle,height(m),breed_category,pet_category


In [51]:
train_df.loc[83]

condition                     1.000000
length(m)                     0.504024
X1                            7.000000
X2                            1.000000
month                        14.133333
                               ...    
color_type_Yellow             0.000000
color_type_Yellow Brindle     0.000000
height(m)                     0.329800
breed_category                0.000000
pet_category                  2.000000
Name: 83, Length: 64, dtype: float64

In [52]:
init_train_df = train_df.drop('pet_category', axis = 1)

In [53]:
init_train_df

,condition,length(m),X1,X2,month,color_type_Agouti,color_type_Apricot,color_type_Black,color_type_Black Brindle,color_type_Black Smoke,...,color_type_Tan,color_type_Torbie,color_type_Tortie,color_type_Tortie Point,color_type_Tricolor,color_type_White,color_type_Yellow,color_type_Yellow Brindle,height(m),breed_category
0,2.0,0.80,13.0,9.0,2.433333,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0778,0.0
1,1.0,0.72,13.0,9.0,62.066667,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0.1419,0.0
2,3.0,0.15,15.0,4.0,25.066667,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.4090,2.0
3,1.0,0.62,0.0,1.0,25.166667,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0.1782,0.0
4,2.0,0.50,18.0,4.0,1.733333,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0.1106,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18829,2.0,0.44,0.0,1.0,13.566667,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0.2736,0.0
18830,3.0,0.73,15.0,4.0,12.866667,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.1425,2.0
18831,0.0,0.99,13.0,9.0,98.466667,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.2813,1.0
18832,0.0,0.55,13.0,9.0,9.066667,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0.4482,1.0


In [54]:
sec_train_df = train_df.drop('breed_category', axis = 1)

In [55]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import f1_score

In [56]:
X1_train = init_train_df.drop('breed_category', axis = 1)
Y1_test = init_train_df['breed_category']

In [57]:
X1_train

,condition,length(m),X1,X2,month,color_type_Agouti,color_type_Apricot,color_type_Black,color_type_Black Brindle,color_type_Black Smoke,...,color_type_Silver Tabby,color_type_Tan,color_type_Torbie,color_type_Tortie,color_type_Tortie Point,color_type_Tricolor,color_type_White,color_type_Yellow,color_type_Yellow Brindle,height(m)
0,2.0,0.80,13.0,9.0,2.433333,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0778
1,1.0,0.72,13.0,9.0,62.066667,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0.1419
2,3.0,0.15,15.0,4.0,25.066667,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.4090
3,1.0,0.62,0.0,1.0,25.166667,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0.1782
4,2.0,0.50,18.0,4.0,1.733333,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.1106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18829,2.0,0.44,0.0,1.0,13.566667,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0.2736
18830,3.0,0.73,15.0,4.0,12.866667,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.1425
18831,0.0,0.99,13.0,9.0,98.466667,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.2813
18832,0.0,0.55,13.0,9.0,9.066667,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0.4482


In [58]:
params = {
        'min_child_weight': [1,2,3,4,5,6,7,8,9,10],
        'gamma': [0.5, 1, 1.5, 2,3,4,5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [1,2,3, 4, 5,6,7],
        
        }

In [59]:
lgm = LGBMClassifier(learning_rate=0.01, n_estimators=800, objective = 'multi:softmax',
                    num_class=3, nthread=1)

In [60]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
folds = 50
param_comb = 50

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 2001)

random_search = RandomizedSearchCV(lgm, param_distributions=params, n_iter=param_comb, n_jobs=4, cv=skf.split(X1_train,Y1_test), verbose=1, random_state=2001 )


In [61]:
random_search.fit(X1_train,Y1_test)

Fitting 50 folds for each of 50 candidates, totalling 2500 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  5.6min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed: 12.3min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 20.6min
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed: 32.6min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed: 46.6min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed: 61.2min
[Parallel(n_jobs=4)]: Done 2500 out of 2500 | elapsed: 62.9min finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x7fafaa48dac0>,
                   error_score=nan,
                   estimator=LGBMClassifier(boosting_type='gbdt',
                                            class_weight=None,
                                            colsample_bytree=1.0,
                                            importance_type='split',
                                            learning_rate=0.01, max_depth=-1,
                                            min_child_samples=20,
                                            min_child_weight=0.001,
                                            min_split_gain=0.0,
                                            n_estimators=800, n_jobs=-1,
                                            nthread=1, num_class=3,
                                            num_leave...
                                            subsample_for_bin=200000,
                                            subsample_freq=0),
                

In [62]:
X2_train = sec_train_df.drop('pet_category', axis = 1)
Y2_test = sec_train_df['pet_category']

In [63]:
params = {
        'min_child_weight': [1,2,3,4,5,6,7,8,9,10],
        'gamma': [0.5, 1, 1.5, 2,3,4,5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [1,2,3, 4, 5,6,7],
        
        }

In [64]:
lgm = LGBMClassifier(learning_rate=0.01, n_estimators=800, objective = 'multi:softmax',
                    num_class=4, nthread=1)

In [65]:
folds = 50
param_comb = 50

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 2001)

random_search_1 = RandomizedSearchCV(lgm, param_distributions=params, n_iter=param_comb, n_jobs=4, cv=skf.split(X2_train,Y2_test), verbose=3, random_state=2001 )


In [ ]:
random_search_1.fit(X2_train,Y2_test)

Fitting 50 folds for each of 50 candidates, totalling 2500 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   52.4s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  3.9min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:  9.8min
[Parallel(n_jobs=4)]: Done 504 tasks      | elapsed: 19.4min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 27.2min
[Parallel(n_jobs=4)]: Done 1144 tasks      | elapsed: 38.0min
[Parallel(n_jobs=4)]: Done 1560 tasks      | elapsed: 49.0min
[Parallel(n_jobs=4)]: Done 2040 tasks      | elapsed: 65.2min


In [ ]:
test_df.head()

In [ ]:
test1 = test_df.drop('pet_id', axis =1)

In [ ]:
test1

In [ ]:
y_test = random_search_1.predict(test1)


In [ ]:
y_test.shape

In [ ]:
test2 = test_df.drop('pet_id', axis =1)

In [ ]:
y_test1 = random_search.predict(test1)

In [ ]:
y_test1

In [ ]:
results_df = pd.DataFrame(data={'pet_id':test_df['pet_id'], 'breed_category':y_test1,'pet_category':y_test})
results_df.to_csv('submission_shuvo2.csv', index=False)